### 代码说明：
* subway.xlsx 地铁线路站点名称及经纬度数据

* graph.pkl 记录站点与站点两两之间的距离

* preprocessing_location.py 封装了get_page_content(request_url)从网页获取地铁站信息、content_analysis()网站内容解析、    get_location(keyword,city)获取经纬度、work()将经纬度插入到subway.xlsx中

* Route_api.py 封装了find_lowest_cost_node(costs)找到最小开销节点、find_shortest_path()找到最短路径、compute(site1,site2) + dijkstra()计算从start到end的最短路径

* main.py 封装了get_nearest_subway(data,longitude1,latitude1)找最近的地铁站、compute(site1,site2,city)计算从site1到site2的最短路径

### 获取地铁站数据

In [1]:
# 获得指定城市的地铁路线
import requests
from bs4 import BeautifulSoup
import pandas as pd

def get_page_content(request_url):
    #得到页面内容
    header={'user-agent': 'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.131 Safari/537.36'}
    html = requests.get(request_url,headers=header,timeout=10)
    content = html.text
    # print(content)
    # 通过content创建BS对象
    # html.parser是bs自带的html解析器
    soup = BeautifulSoup(content,'html.parser',from_encoding='utf-8')
    return soup
    
request_url = 'https://ditie.mapbar.com/beijing_line/'
soup = get_page_content(request_url)

# 解析内容
subways = soup.find_all('div',class_='station')
df = pd.DataFrame(columns=['name','site'])
for subway in subways:
    # 得到线路名称
    route_name = subway.find('strong',class_='bolder').text
    # print('routename = ',route_name)
    # 找到该线路中每一站的名称
    routes = subway.find('ul')
    routes = routes.find_all('a')
    for route in routes:
        # name 地铁站名 site 线路名
        temp = {'name':route.text,'site':route_name}
        # print('route = ',route.text)
        df = df.append(temp,ignore_index=True)
        
# print(df)        
df['city'] = '北京'
df.to_excel('./subway.xlsx',index=False)
        

C:\Users\admin\AppData\Roaming\Python\Python36\site-packages\bs4\__init__.py:203: UserWarning: You provided Unicode markup but also provided a value for from_encoding. Your from_encoding will be ignored.
  warnings.warn("You provided Unicode markup but also provided a value for from_encoding. Your from_encoding will be ignored.")


### 获取经纬度

In [4]:
import re

# 获取经度longitude 纬度latitude
def get_location(keyword,city):
    header={'user-agent': 'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.131 Safari/537.36'}
    request_url = 'http://restapi.amap.com/v3/place/text?key=f839aebd38cd68cc4802c99b45298674&keywords=' + keyword + '&types=&city=' + city + '&children=1&offset=1&page=1&extensions=all'
    data = requests.get(request_url,headers=header)
    data.encoding = 'utf-8'
    data = data.text
    # print(data)
    # "location":"116.337581,39.993138"
    # .*具有贪婪的性质，首先匹配到不能匹配为止
    # 。*？ 则相反，一个匹配以后，就可以继续后面的匹配
    pattern = 'location":"(.*?),(.*?)"'
    # 获取经纬度
    '''
        IndexError                                Traceback (most recent call last)
    <ipython-input-9-dee0fef02142> in <module>
         22 df['longitude'],df['latitude'] = None, None
         23 for index,row in df.iterrows():
    ---> 24     longitude,latitude = get_location(row['name'],row['city'])
         25     df.iloc[index]['longitude'] = longitude
         26     df.iloc[index]['latitude'] = latitude

    <ipython-input-9-dee0fef02142> in get_location(keyword, city)
         15     # 获取经纬度
         16     result = re.findall(pattern,data)
    ---> 17     return result[0][0],result[0][1]
         18 
         19 get_location('五道口站','北京')

    IndexError: list index out of range
    
    此错误是由于高德地图API中没有"石门站" 而是"石门"
    所以下面要加try except
    '''
    result = re.findall(pattern,data)
    try:
        return result[0][0],result[0][1]
    except:
        return get_location(keyword.replace('站',''),city)   

df = pd.read_excel('./subway.xlsx')
df['longitude'],df['latitude'] = None, None
for index,row in df.iterrows():
    longitude,latitude = get_location(row['name'],row['city'])
    df.iloc[index]['longitude'] = longitude
    df.iloc[index]['latitude'] = latitude
    #print(longitude,latitude)

get_location('五道口站','北京')
df.to_excel('./subway.xlsx',index=False)   

### 进行路径规划   

In [12]:
# 获取数据
data = pd.read_excel('./subway.xlsx')
print(data)


      name    site city   longitude   latitude
0      四惠站   地铁八通线   北京  116.498489  39.908279
1     四惠东站   地铁八通线   北京  116.514954  39.908051
2     高碑店站   地铁八通线   北京  115.858901  39.328727
3    传媒大学站   地铁八通线   北京  116.554931  39.909548
4      双桥站   地铁八通线   北京  116.569498  39.901066
..     ...     ...  ...         ...        ...
346    永丰站  地铁16号线   北京  116.237766  40.071247
347   永丰南站  地铁16号线   北京  116.247876  40.064807
348   西北旺站  地铁16号线   北京  116.258460  40.048344
349   马连洼站  地铁16号线   北京  116.274073  40.032358
350    西苑站  地铁16号线   北京  116.291136  39.997817

[351 rows x 5 columns]


In [15]:
from collections import defaultdict

# 计算两点之间的距离
def compute_distance(longitude1,latitude1,longitude2,latitude2):
    header={'user-agent': 'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.131 Safari/537.36'}
    request_url = 'http://restapi.amap.com/v3/distance?key=f839aebd38cd68cc4802c99b45298674&origins=' + str(longitude1) + ',' + str(latitude1) + '&destination='  + str(longitude2) + ',' + str(latitude2) + '&type=1'
    data = requests.get(request_url,headers=header)
    data.encoding = 'utf-8'
    data = data.text
    # print(data)
    # "distance":"2646","duration":"540"
    pattern = 'distance":"(.*?)","duration":"(.*?)"'
    result = re.findall(pattern,data)
    #print(result[0][0],result[0][1])
    return result[0][0]
    
# 保存途中两点之间的距离
graph = defaultdict(dict)

for i in range(data.shape[0]):
    site1 = data.iloc[i]['site']
    if i < data.shape[0]-1:
        site2 = data.iloc[i+1]['site']
        # 如果是同一条线路
        if site1 == site2:
            longitude1,latitude1 = data.iloc[i]['longitude'],data.iloc[i]['latitude']
            longitude2,latitude2 = data.iloc[i+1]['longitude'],data.iloc[i+1]['latitude']
            name1 = data.iloc[i]['name']
            name2 = data.iloc[i+1]['name'] 
            # 按照距离，计算两点之间距离
            distance = compute_distance(longitude1,latitude1,longitude2,latitude2)
            graph[name1][name2] = distance
            graph[name2][name1] = distance
            print(name1,name2,distance)
            
import pickle
output = open('graph.pkl','wb')
pickle.dump(graph,output)


            
#compute_distance(116.337581,39.993138,116.339941,39.976228)            

四惠站 四惠东站 4143
四惠东站 高碑店站 102121
高碑店站 传媒大学站 111741
传媒大学站 双桥站 4827
双桥站 管庄站 3607
管庄站 八里桥站 3242
八里桥站 通州北苑站 3409
通州北苑站 果园站 1472
果园站 九棵树站 1510
九棵树站 梨园站 2323
梨园站 临河里站 2727
临河里站 土桥站 1674
西二旗站 生命科学园站 6812
生命科学园站 朱辛庄站 3479
朱辛庄站 巩华城站 6679
巩华城站 沙河站 6498
沙河站 沙河高教园站 3845
沙河高教园站 南邵站 6825
南邵站 北邵洼站 1981
北邵洼站 昌平东关站 2741
昌平东关站 昌平站 9602
昌平站 十三陵景区站 9552
十三陵景区站 昌平西山口站 2093
苏庄站 良乡南关站 2180
良乡南关站 良乡大学城西站 2266
良乡大学城西站 良乡大学城站 2707
良乡大学城站 良乡大学城北站 1514
良乡大学城北站 广阳城站 2767
广阳城站 篱笆房站 1243
篱笆房站 长阳站 2237
长阳站 稻田站 4280
稻田站 大葆台站 15844
大葆台站 郭公庄站 1951
宋家庄站 肖村站 3417
肖村站 小红门站 2201
小红门站 旧宫站 5879
旧宫站 亦庄桥站 3397
亦庄桥站 亦庄文化园站 2001
亦庄文化园站 万源街站 2362
万源街站 荣京东街站 1537
荣京东街站 荣昌东街站 1465
荣昌东街站 同济南路站 2340
同济南路站 经海路站 2331
经海路站 次渠南站 3275
次渠南站 次渠站 2001
东直门站 三元桥站 3133
三元桥站 三号航站楼站 21939
三号航站楼站 二号航站楼站 6395
二号航站楼站 三元桥站 19870
三元桥站 东直门站 3312
苹果园站 古城站 4650
古城站 八角游乐园站 3191
八角游乐园站 八宝山站 2190
八宝山站 玉泉路站 3591
玉泉路站 五棵松站 1734
五棵松站 万寿路站 2346
万寿路站 公主坟站 2217
公主坟站 军事博物馆站 825
军事博物馆站 木樨地站 4988
木樨地站 南礼士路站 2051
南礼士路站 复兴门站 1819
复兴门站 西单站 4441
西单站 天安门西站 5660
天安门西站 天安门东站 

In [16]:
# 读取pickle数据
file = open('graph.pkl','rb')
graph = pickle.load(file)
print(graph)

defaultdict(<class 'dict'>, {'四惠站': {'四惠东站': '4143', '大望路站': '4415'}, '四惠东站': {'四惠站': '4143', '高碑店站': '102121'}, '高碑店站': {'四惠东站': '102121', '传媒大学站': '111741'}, '传媒大学站': {'高碑店站': '111741', '双桥站': '4827'}, '双桥站': {'传媒大学站': '4827', '管庄站': '3607'}, '管庄站': {'双桥站': '3607', '八里桥站': '3242'}, '八里桥站': {'管庄站': '3242', '通州北苑站': '3409'}, '通州北苑站': {'八里桥站': '3409', '果园站': '1472'}, '果园站': {'通州北苑站': '1472', '九棵树站': '1510'}, '九棵树站': {'果园站': '1510', '梨园站': '2323'}, '梨园站': {'九棵树站': '2323', '临河里站': '2727'}, '临河里站': {'梨园站': '2727', '土桥站': '1674'}, '土桥站': {'临河里站': '1674'}, '西二旗站': {'生命科学园站': '6812', '上地站': '2862', '龙泽站': '6892'}, '生命科学园站': {'西二旗站': '6812', '朱辛庄站': '3479'}, '朱辛庄站': {'生命科学园站': '3479', '巩华城站': '6679', '育知路站': '2964'}, '巩华城站': {'朱辛庄站': '6679', '沙河站': '6498'}, '沙河站': {'巩华城站': '6498', '沙河高教园站': '3845'}, '沙河高教园站': {'沙河站': '3845', '南邵站': '6825'}, '南邵站': {'沙河高教园站': '6825', '北邵洼站': '1981'}, '北邵洼站': {'南邵站': '1981', '昌平东关站': '2741'}, '昌平东关站': {'北邵洼站': '2741', '昌平站': '9602'}, '昌平站': {'昌平东关站': '9602', '十三

In [25]:
# 找到最小开销节点(从U中)
def find_lowest_cost_node(costs):
    #初始化数据
    lowest_cost = float('inf') # 打擂法，初始最小值为正无穷
    lowest_cost_node = None
    # 遍历所有节点
    for node in costs:
        #如果该节点没有被处理
        if not node in processed:
            # 如果当前节点的开销比已经存在的开销小，那么就更新该节点为开销最小节点
            if costs[node] < lowest_cost:
                lowest_cost = costs[node]
                lowest_cost_node = node
    return lowest_cost_node            

# 找到最短路径
def find_shortest_path():
    node = end
    shortest_path = [end]
    # 最终的根节点为start
    while parents[node] != start:
        # 往前移动一步
        node = parents[node]
        # 添加到路径中
        shortest_path.append(node)
    shortest_path.append(start)
        
    return shortest_path

# 计算图中从start到end的最短路径
def dijkstra():
    # 查询到目前开销最小节点
    node = find_lowest_cost_node(costs)
    # print('当前cost最小节点：',node)
    # 使用找到的开销最小的节点，计算它的令居，是否可以通过它进行更新
    # 如果所有的节点都在processed里面，就结束
    while node is not None:
        # 获取节点的cost
        cost = costs[node] # cost是从node到start
        # 获取节点的邻居
        neighbors = graph[node]
        # 遍历所有邻居，看是否可以通过它进行更新
        for neighbor in neighbors.keys():
            # 计算邻居到当前节点 + 当前节点开销
            new_cost = cost + float(neighbors[neighbor])
            if neighbor not in costs or new_cost < costs[neighbor]:
                costs[neighbor] = new_cost
                # 经过node到达neighbor节点，cost更少
                parents[neighbor] = node
        # 将当前节点标记为已处理
        processed.append(node)
        # 下一步，继续找U中最短距离的节点 costs=U,processed=S
        node = find_lowest_cost_node(costs)
        # print('当前cost最小节点：',node)
    
    # 循环完，说明所有节点已经处理完
    shortest_path = find_shortest_path()
    shortest_path.reverse()
    print('从{}到{}的最短路径：{}'.format(start,end,shortest_path))
start = '五道口站'
end = '北京南站'
# 创建节点的开销表，cost是指从start到该节点的距离
costs = {}
# 存储父节点的Hash表，用于记录路径
parents = {} 
parents[end] = None

# 获取节点相邻的节点
#print(graph[start].keys())  dict_keys(['知春路站', '上地站'])
for node in graph[start].keys():
    costs[node] = float(graph[start][node])
    parents[node] = start
#终点到起始点设置为无穷大
costs[end] = float('inf')

# 记录处理过的节点list
processed = []

dijkstra()


从五道口站到北京南站的最短路径：['五道口站', '知春路站', '大钟寺站', '西直门站', '新街口站', '平安里站', '西四站', '灵境胡同站', '西单站', '宣武门站', '菜市口站', '陶然亭站', '北京南站']


In [28]:
%run Route_api.py

['五道口站', '知春路站', '大钟寺站', '西直门站', '新街口站', '平安里站', '西四站', '灵境胡同站', '西单站', '宣武门站', '菜市口站', '陶然亭站', '北京南站']


###  在main.py中需要进行以下设置才可以通过命令行传参的方式运行程序
#下面这个部分是为了在命令行或者Cell中执行命令 程序可以知道参数要传到哪

import argparse 

parser = argparse.ArgumentParser()   

parser.add_argument('--site1', type=str, default='清华大学', help='site1')

parser.add_argument('--site2', type=str, default='798', help='site2')

parser.add_argument('--city', type=str, default='北京', help='city')

args = parser.parse_known_args()[0]

site1=args.site1

site2=args.site2

city=args.city

In [39]:
%run main.py --site1 清华大学 --site2 798 --city 北京

清华大学最近的地铁站为五道口站
798最近的地铁站为将台站
从清华大学 => 798的最短路径为: '清华大学', '五道口站', '知春路站', '西土城站', '牡丹园站', '健德门站', '北土城站', '安贞门站', '惠新西街南口站', '芍药居站', '望京西站', '望京站', '阜通站', '望京南站', '将台站', '798'
